## File type demo

This notebook demonstrates some of the capabilities of the eegyolk library in terms of reading the cnt file type. It also demonstrates the importance of the dataformat argument in the mne library. Previous work had let some arguments in reading cnt files default to preset values.
This notebook specifically demonstrates how whether the format of int32 or int16 influences the understanding of the file

#### Imports

In [1]:
import mne      # toolbox for analyzing and visualizing EEG data
import os       # using operating system dependent functionality (folders)
import pandas   # data analysis and manipulation
import numpy    # numerical computing (manipulating and performing operations on arrays of data)
import copy     # Can Copy and Deepcopy files so original file is untouched.
import glob
import numpy as np
import pandas as pd
from numpy.fft import fft, fftfreq
from scipy import signal

import matplotlib
import matplotlib.pyplot as plt

from mne.time_frequency.tfr import morlet
from mne.viz import plot_filter, plot_ideal_filter


from IPython.display import clear_output
import warnings

import sys

Below eegyolk is importted in it's most updated version
But the cell could be replaced by importing the stable library and then importing the modules of it.

In [2]:

import sys
sys.path.insert(0, '../eegyolk') # path to helper functions
import helper_functions as hf # library useful for eeg and erp data cleaning
#import initialization_functions #library to import data
import epod_helper
import raw
from config import Config
from raw import RawData

#### Load EEG files
Change your config file to change where data is coming from

In [3]:
!pwd

/home/cmoore/eegyolk/demos


In [4]:
config = Config()


Object `SwitchedRawData` not found.


In [6]:
#eeg_file_root = "../../volume-ceph/ePodium_projectfolder"
acquired = RawData(config.get_directory('data'), config.get_directory('metadata'))

Note we assume you are looking at bdf files

In [7]:
acquired.raw.head()

,code,cnt_path,cnt_file,age_group,age_days,age_months,age_years
0,35,/volume-ceph/DDP_projectfolder/11mnd mmn/035_1...,035_11_jc_mmn36_slp_mmn25_slp,11,331.0,11.033333,0.919444
1,27,/volume-ceph/DDP_projectfolder/11mnd mmn/027_1...,027_11_jc_mmn25_wk,11,326.0,10.866667,0.905556
2,25,/volume-ceph/DDP_projectfolder/11mnd mmn/025_1...,025_11_mc_mmn36_wk,11,360.0,12.000000,1.000000
3,35,/volume-ceph/DDP_projectfolder/11mnd mmn/035_1...,035_11_jc_mmn36slp_mmn25_slp_2,11,331.0,11.033333,0.919444
4,30,/volume-ceph/DDP_projectfolder/11mnd mmn/030_1...,030_11_jc_mmn36_wk_mmn25_wk,11,328.0,10.933333,0.911111


In [8]:
data_raw = acquired.as_mne[2]

In [9]:
print('Data type: {}\n\n{}\n'.format(type(data_raw), data_raw))

# Get the sample rate
print('Sample rate:', data_raw.info['sfreq'], 'Hz')

# Get the size of the matrix
print('Size of the matrix: {}\n'.format(data_raw.get_data().shape))

# The mne.info class can be used to learn more about the data.
print(data_raw.info)

Data type: <class 'mne.io.cnt.cnt.RawCNT'>

<RawCNT | 025_11_mc_mmn36_wk.cnt, 64 x 195390 (390.8 s), ~95.5 MB, data loaded>

Sample rate: 500.0 Hz
Size of the matrix: (64, 195390)

<Info | 8 non-empty values
 bads: []
 ch_names: O2, O1, OZ, PZ, P4, CP4, P8, C4, TP8, T8, P7, P3, CP3, CPZ, CZ, ...
 chs: 62 EEG, 2 EOG
 custom_ref_applied: False
 highpass: 0.0 Hz
 lowpass: 250.0 Hz
 meas_date: 2002-03-19 11:50:16 UTC
 nchan: 64
 projs: []
 sfreq: 500.0 Hz
 subject_info: 5 items (dict)
>


# Now we will show as a pandas dataframe

In [10]:
raw_df = data_raw.to_data_frame()
raw_df.head()

,time,O2,O1,OZ,PZ,P4,CP4,P8,C4,TP8,...,F2,F6,FC5,F1,AF4,AF8,F5,AF7,AF3,FPZ
0,0,6.253748e+06,8.242944e+06,1.500095e+07,2.436091e+07,1.069054e+07,1.188921e+06,3.539428e+06,1.705889e+07,-2.513232e+06,...,-4.592944e+06,3.691768e+06,1.293685e+07,2.131507e+07,7.617701e+06,5.630349e+06,7.997737e+06,5.734013e+06,7.490854e+05,8.542372e+06
1,2,7.166750e+06,9.477653e+06,1.618734e+07,2.542505e+07,1.253922e+07,1.337556e+06,4.167398e+06,1.810656e+07,-8.982657e+05,...,-3.960995e+06,4.459465e+06,1.381143e+07,2.213357e+07,8.093100e+06,6.695248e+06,8.453425e+06,6.465057e+06,4.346442e+06,1.016948e+07
2,4,6.961324e+06,9.260436e+06,1.563977e+07,2.508177e+07,1.163209e+07,1.063176e+06,3.916206e+06,1.789019e+07,-1.182595e+06,...,-5.777863e+06,2.551495e+06,1.189189e+07,2.021237e+07,6.451839e+06,4.973270e+06,6.778696e+06,4.169124e+06,-3.177539e+05,8.372844e+06
3,6,4.096767e+06,6.105062e+06,1.225170e+07,2.168337e+07,8.577715e+06,-1.349102e+06,1.404325e+06,1.465605e+07,-5.129023e+06,...,-9.208469e+06,-1.083805e+06,8.166398e+06,1.655184e+07,3.271193e+06,1.370719e+06,3.622926e+06,2.511902e+05,-6.650079e+06,4.553613e+06
4,8,2.031094e+06,4.047205e+06,1.012988e+07,1.947497e+07,6.683092e+06,-3.052541e+06,-2.969035e+05,1.258346e+07,-6.982832e+06,...,-9.569584e+06,-1.388626e+06,7.666639e+06,1.606302e+07,2.908989e+06,8.042842e+05,3.326748e+06,4.453684e+05,-8.068571e+06,2.350257e+06


Now we used the default settings above, let's just switch the settings and compare

In [11]:
def read_raw_agnostic(fname, int_arg):
    """
    For testing purposes
    """
    
    cnt_read_args = {
        'eog': 'auto',
        'data_format': int_arg,
        'date_format': 'dd/mm/yy',
        'verbose': False,
    }
    read = mne.io.read_raw_cnt(
            fname,
            data_format=int_arg,
            preload=True,
            
        )
    return read

In [12]:
n = 2

In [13]:
paths_df = acquired.raw
paths_df.cnt_path[n]

'/volume-ceph/DDP_projectfolder/11mnd mmn/025_11_mc_mmn36_wk.cnt'

In [14]:
read_on_32 = read_raw_agnostic(paths_df.cnt_path[n], 'int32')


Reading 0 ... 195389  =      0.000 ...   390.778 secs...


/tmp/ipykernel_3090738/3019677501.py:12: RuntimeWarning:   Could not parse meas date from the header. Setting to None.
  read = mne.io.read_raw_cnt(


In [15]:
raw_df32 = read_on_32.to_data_frame()
raw_df32.head()

,time,O2,O1,OZ,PZ,P4,CP4,P8,C4,TP8,...,F2,F6,FC5,F1,AF4,AF8,F5,AF7,AF3,FPZ
0,0,6.253748e+06,8.242944e+06,1.500095e+07,2.436091e+07,1.069054e+07,1.188921e+06,3.539428e+06,1.705889e+07,-2.513232e+06,...,-4.592944e+06,3.691768e+06,1.293685e+07,2.131507e+07,7.617701e+06,5.630349e+06,7.997737e+06,5.734013e+06,7.490854e+05,8.542372e+06
1,2,7.166750e+06,9.477653e+06,1.618734e+07,2.542505e+07,1.253922e+07,1.337556e+06,4.167398e+06,1.810656e+07,-8.982657e+05,...,-3.960995e+06,4.459465e+06,1.381143e+07,2.213357e+07,8.093100e+06,6.695248e+06,8.453425e+06,6.465057e+06,4.346442e+06,1.016948e+07
2,4,6.961324e+06,9.260436e+06,1.563977e+07,2.508177e+07,1.163209e+07,1.063176e+06,3.916206e+06,1.789019e+07,-1.182595e+06,...,-5.777863e+06,2.551495e+06,1.189189e+07,2.021237e+07,6.451839e+06,4.973270e+06,6.778696e+06,4.169124e+06,-3.177539e+05,8.372844e+06
3,6,4.096767e+06,6.105062e+06,1.225170e+07,2.168337e+07,8.577715e+06,-1.349102e+06,1.404325e+06,1.465605e+07,-5.129023e+06,...,-9.208469e+06,-1.083805e+06,8.166398e+06,1.655184e+07,3.271193e+06,1.370719e+06,3.622926e+06,2.511902e+05,-6.650079e+06,4.553613e+06
4,8,2.031094e+06,4.047205e+06,1.012988e+07,1.947497e+07,6.683092e+06,-3.052541e+06,-2.969035e+05,1.258346e+07,-6.982832e+06,...,-9.569584e+06,-1.388626e+06,7.666639e+06,1.606302e+07,2.908989e+06,8.042842e+05,3.326748e+06,4.453684e+05,-8.068571e+06,2.350257e+06


In [16]:
read_on_16 = read_raw_agnostic(paths_df.cnt_path[n], 'int16')
raw_df16 = read_on_16.to_data_frame()
raw_df16.head()

Reading 0 ... 390779  =      0.000 ...   781.558 secs...


/tmp/ipykernel_3090738/3019677501.py:12: RuntimeWarning:   Could not parse meas date from the header. Setting to None.
  read = mne.io.read_raw_cnt(


,time,O2,O1,OZ,PZ,P4,CP4,P8,C4,TP8,...,F2,F6,FC5,F1,AF4,AF8,F5,AF7,AF3,FPZ
0,0,-274.444902,95.420848,211.836368,125.883886,104.424014,229.393813,166.724743,369.939166,382.993497,...,21.351700,75.968990,126.863087,115.004328,-40.587220,75.540190,107.605393,-40.784313,80.346320,118.103448
1,2,-271.136239,103.968602,220.539587,137.058045,115.462123,241.256002,171.777008,379.843596,390.802608,...,30.650021,85.615846,122.357021,121.769288,-29.706391,86.603284,102.911781,11.503268,94.199134,130.344827
2,4,-268.524137,109.376365,226.631841,144.740279,121.243989,247.535985,174.738680,386.099026,390.629072,...,37.193284,95.262702,113.171579,126.279262,-21.934370,94.036300,96.132120,52.287580,102.164502,145.862068
3,6,-266.086174,112.341913,230.287193,148.406800,122.996070,248.931537,175.087112,389.052979,386.117142,...,40.292725,101.808783,106.239170,128.707709,-18.825562,97.666378,71.621037,66.753811,100.779221,155.172413
4,8,-270.962099,106.236374,224.717133,141.422950,114.410874,239.162675,167.770039,380.886168,374.490243,...,32.716315,96.640824,100.866553,122.289670,-27.288429,88.677614,33.202956,44.444443,86.406926,148.275862


In [17]:
# The big difference

In [18]:
(raw_df32 -raw_df16).sum().sum()

-37583506229279.79

### Youch!

So here we saw an example of how reading the same file on int32 and int16 will give us dramatically different results. We must know which we have when we use cnt files.

## This is no floating point error!

So is there anything we can do to figure out which kind of files we had?
Why, yes of course. First of all on a few files in the DDP dataset there is information on this
in the metadata (numchannels). Second of all, we can be a bit sneaky, and ask ourselves if
32 is a possibility. When we look at the number of bit and bytes inside the file for the
information, is this a number that we could divide by 32? Well it turns out when you peel the header off of
the files you can calculate the number of bits/bytes inside. And they are (all tested so far) with a bit
number divisible by 32 (or a byte number divisible by 4). But wait, that means de-facto , they
are also divisble by 16. So we can say probably the files should have been read as 32; but
we can not say with 100% certainty. This is not a problem outside the cnt format, but for cnt formatted
data, back to cell 1.

In [1]:
4*8

32